In [4]:
train_proteins = []
train_peptides = []
val_proteins = []
val_peptides = []

with open('/content/8_14_24_sequence_pairs_ALL.txt', 'r') as f:
  for line in f.readlines():
    pro, pep, split = line.split(':')
    if '0' in split:
      train_proteins.append(pro)
      train_peptides.append(pep)
    else:
      val_proteins.append(pro)
      val_peptides.append(pep)

In [5]:
print(len(train_proteins))
print(len(train_peptides))
print(len(val_proteins))
print(len(val_peptides))

9668
9668
1118
1118


In [8]:
import pandas as pd
train_df = pd.DataFrame({'peptide': train_peptides, 'protein': train_proteins})
val_df = pd.DataFrame({'peptide': val_peptides, 'protein': val_proteins})

In [9]:
selected_data_unique = train_df

# Generate synthetic negative data by shuffling the 'Core' column
shuffled_cores = selected_data_unique['peptide'].sample(frac=1, random_state=12).reset_index(drop=True)

# Create a new DataFrame with the original 'Cyclase sequence' and the shuffled 'Core'
negative_data = pd.DataFrame({
  'protein': selected_data_unique['protein'],
  'peptide': shuffled_cores
})

# Ensure there are no duplicates in the synthetic negative data compared to the original positive data
negative_data_unique = negative_data.drop_duplicates()
# negative_data_unique.to_csv('negative_cyclase_data.csv', index = False)

# print(negative_data_unique)
# Find common rows
common_rows = pd.merge(selected_data_unique, negative_data_unique, on=['protein', 'peptide'], how='inner')

# Print the common rows
# print("Number of common rows:", common_rows.shape[0])
# print(common_rows)

# Remove common rows from negative_data_unique
negative_data_unique_clean = negative_data_unique[~negative_data_unique.apply(tuple, axis = 1).isin(common_rows.apply(tuple, axis=1))]
# print(negative_data_unique_clean)

combined_data = pd.concat([selected_data_unique, negative_data_unique_clean], ignore_index= True)
# print(combined_data.shape)

combined_data['label'] = selected_data_unique.shape[0] * [1] + negative_data_unique_clean.shape[0] * [0]
combined_data.to_csv('training_pairs_spec.csv', index = False)

In [13]:
selected_data_unique = val_df

# Generate synthetic negative data by shuffling the 'Core' column
shuffled_cores = selected_data_unique['peptide'].sample(frac=1, random_state=12).reset_index(drop=True)

# Create a new DataFrame with the original 'Cyclase sequence' and the shuffled 'Core'
negative_data = pd.DataFrame({
  'protein': selected_data_unique['protein'],
  'peptide': shuffled_cores
})

# Ensure there are no duplicates in the synthetic negative data compared to the original positive data
negative_data_unique = negative_data.drop_duplicates()
# negative_data_unique.to_csv('negative_cyclase_data.csv', index = False)

# print(negative_data_unique)
# Find common rows
common_rows = pd.merge(selected_data_unique, negative_data_unique, on=['protein', 'peptide'], how='inner')

# Print the common rows
# print("Number of common rows:", common_rows.shape[0])
# print(common_rows)

# Remove common rows from negative_data_unique
negative_data_unique_clean = negative_data_unique[~negative_data_unique.apply(tuple, axis = 1).isin(common_rows.apply(tuple, axis=1))]
# print(negative_data_unique_clean)

combined_data = pd.concat([selected_data_unique, negative_data_unique_clean], ignore_index= True)
# print(combined_data.shape)

combined_data['label'] = selected_data_unique.shape[0] * [1] + negative_data_unique_clean.shape[0] * [0]
combined_data.to_csv('val_pairs_spec.csv', index = False)

In [14]:
df = pd.read_csv('val_pairs_spec.csv')
sum(df['label'].tolist())/len(df['label'].tolist())

0.5